In [0]:
files = dbutils.fs.ls('/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/')

for fi in files: 
     print(fi.path)

dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/1/
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/1917.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/47_Meters_Down_Uncaged.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/A_Beautiful_Day_in_the_Neighborhood.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/A_Dogs_Journey.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/A_Dogs_Way_Home.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/A_Madea_Family_Funeral.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/Ad_Astra.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/After.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/Alita_Battle_Angel.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/Angel_Has_Fallen.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/Annabelle_Comes_Home.csv
dbfs:/FileStore/shared_uploads/isaiah.williams@ucalgary

In [0]:
dbutils.fs.rm("/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/xlsxYoutubeData_and_MovieMetaData__1_.csv")

Out[3]: False

In [0]:
!pip install nltk
!pip install wordninja
!pip install contractions
!pip install emoji

     |████████████████████████████████| 1.5 MB 6.8 MB/s 
     |████████████████████████████████| 96 kB 259 kB/s 
     |████████████████████████████████| 772 kB 45.6 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 541 kB 5.5 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541553 sha256=2cbae1ae362a8e00cb5fa09a44903dc60db3065941196ae167ec6b2510e27bd1
  Stored in directory: /root/.cache/pip/wheels/a6/08/fb/98831d1c2702c8352a339f91104515eab5d7906d4118eeeaed
Successfully built wordninja
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 110 kB 7.0 MB/s 
     |████████████████████████████████| 287 kB 43.6 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upg

In [0]:
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Out[5]: True

In [0]:
import pyspark.sql    
from pyspark.sql import SparkSession    
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
import emoji
import contractions
import string
from nltk.corpus import words
import nltk
import re
import wordninja
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
@udf
def lowerCase(comment):
    return comment.lower()

@udf
def convertEmoji(comment):
    return emoji.demojize(comment).replace(":"," ").replace("_", " ")

@udf
def convertContractions(comment):
    return contractions.fix(comment, slang=True)

@udf 
def removePunctuation(comment):
    return ' '.join(''.join("" if c in list(string.punctuation) else c for c in y) for y in comment.split())

@udf
def removeNumbers(comment):
    return re.sub(r'[0-9]+', '', comment)
    
@udf
def filterWords(comment):
    if ((comment is None) | (comment =="")):
        return None
    word_list=nltk.corpus.words.words()
    words = [1 if word in word_list else 0 for word in comment.split()]
    if ((len(words) == 1) |((np.sum(words)/len(words))>0.6)):
        return comment
    return None

@udf 
def inferSpaces(comment):  
    return ' '.join(wordninja.split(comment))

@udf
def findLema(comment):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

@udf 
def filterWordLength(comment):
    return ' '.join(['' if len(c)<3 else c for c in str(comment).split()])

# sqlContext.registerDataFrameAsTable(unanswered, "table1")
#         unanswered = spark.sql("SELECT * FROM table1 t WHERE t.Score = (SELECT max(a.Score) FROM table1 a) LIMIT 1")


In [0]:
class CommentSentiment(object):
    def __init__(self):
        pass
    
    def readFile(self, file_path):
        df = spark.read.option("header", True).option("wholeFile", True).option("escape", "\"").option("multiLine", True).csv(file_path)
        df = self.preProcess(df)
        return df
 
    def createAllMovieDf(self,folder_path):
        files = dbutils.fs.ls(folder_path)
        for i,fi in enumerate(files): 
            if i == 0:
                df = spark.read.option("header", True).option("wholeFile", True).option("escape", "\"").option("multiLine", True).csv(fi.path)
            else:
                df = df.unionByName(spark.read.option("header", True).option("wholeFile", True).option("escape", "\"").option("multiLine", True).csv(fi.path), allowMissingColumns=True)
        return df

    def preProcess(self,df): 
        df = df.withColumn('Comment', lowerCase('Comment'))
        df = df.withColumn('Comment', convertEmoji('Comment'))
        df = df.withColumn('Comment', removePunctuation('Comment'))
        df = df.withColumn('Comment', removeNumbers('Comment'))
        df = df.withColumn('Comment', filterWords('Comment'))
        df = df.filter(df.Comment.isNotNull())
        df = df.withColumn('Comment', inferSpaces('Comment'))
        df = df.withColumn('Comment', findLema('Comment'))
        df = df.withColumn('Comment', filterWordLength('Comment'))
        return df

In [0]:
cs = CommentSentiment()
trainingFile = cs.readFile('/FileStore/labelled_data_1.csv')



In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import NGram
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
from pyspark.mllib.evaluation import MulticlassMetrics
import pandas as pd
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:

indexer = StringIndexer(inputCol="Final Sentiment Decision", outputCol="Encoded Sentiment Decision") 
df = indexer.fit(trainingFile).transform(trainingFile) 
df2 = df.select("Comment","Encoded Sentiment Decision")\
        .withColumnRenamed("Encoded Sentiment Decision", "label")
df2= df2.dropna()

In [0]:
train, test = df2.randomSplit([0.80, 0.20], 1234)
train.cache()
test.cache()

Out[11]: DataFrame[Comment: string, label: double]

In [0]:
tk = Tokenizer(inputCol= "Comment", outputCol = "tokens")
gram = NGram(inputCol="tokens", outputCol="gram")
tf     = HashingTF(inputCol="gram", outputCol="rawFeatures", numFeatures=2e5)
idf     = IDF(inputCol="rawFeatures", outputCol="features")
lr      = LogisticRegression(maxIter=20)

pipeline  = Pipeline(stages= [tk, gram, tf, idf, lr])
paramGrid = ParamGridBuilder() \
                        .addGrid(gram.n, [1,2,3]) \
                        .addGrid(idf.minDocFreq, [1,2, 5]) \
                        .addGrid(lr.regParam, [0.0, 0.1]) \
                        .addGrid(lr.elasticNetParam, [0,1]) \
                        .build()
crossval = CrossValidator(estimator          = pipeline,
                          estimatorParamMaps = paramGrid,
                          evaluator          = MulticlassClassificationEvaluator(),
                          numFolds           = 2,
                          collectSubModels=True)
model = crossval.fit(train)

In [0]:
def hyperparameter_getter(model_obj,cv_fold = 2.0):
    enet_list = []
    reg_list  = []
    n_list  = []
    minDocFreq_list  = []

    metrics = model_obj.avgMetrics
    assert type(metrics) is list
    assert len(metrics) > 0

    for x in range(len(model_obj._paramMap.keys())):
        if list(model_obj._paramMap.keys())[x].name=='estimatorParamMaps':
            param_map_key = list(model_obj._paramMap.keys())[x]

    params = model_obj._paramMap[param_map_key]

    for i in range(len(params)):
        for k in params[i].keys():
            if k.name =='n':
                n_list.append(params[i][k])
            if k.name =='regParam':
                reg_list.append(params[i][k])
            if k.name =='minDocFreq':
                minDocFreq_list.append(params[i][k])
            if k.name =='elasticNetParam':
                enet_list.append(params[i][k])

    results_df =  pd.DataFrame({
             'model':'logistic_regression',
             'minDocFreq': minDocFreq_list, 
             'n': n_list, 
             'regParam':reg_list,
             'elasticNetParam':enet_list,
             'metrics':metrics})

    spark_version = [int(x) for x in sc.version.split('.')]

    if spark_version[0] <= 2:
        if spark_version[1] < 1:
            results_df.metrics = 1.0*results_df['metrics'] / cv_fold

    return results_df
trainingResults = hyperparameter_getter(model)

In [0]:
tk = Tokenizer(inputCol= "Comment", outputCol = "tokens")
gram = NGram(inputCol="tokens", outputCol="grams")
tf     = HashingTF(inputCol="grams", outputCol="rawFeatures", numFeatures=2e5)
idf     = IDF(inputCol="rawFeatures", outputCol="features")
nb      = NaiveBayes()

naive_bayes_pipeline  = Pipeline(stages= [tk, gram, tf, idf, nb])
paramGrid_nb = ParamGridBuilder() \
                        .addGrid(gram.n, [1,2,3]) \
                        .addGrid(idf.minDocFreq, [1, 2, 5]) \
                        .addGrid(nb.smoothing, [0.0, 0.5,1.0]) \
                        .build()
crossval = CrossValidator(estimator          = naive_bayes_pipeline,
                          estimatorParamMaps = paramGrid_nb,
                          evaluator          = MulticlassClassificationEvaluator(),
                          numFolds           = 2, collectSubModels=True)
model_nb    = crossval.fit(train)

In [0]:
def hyperparameter_getter(model_obj,cv_fold = 2.0):
    enet_list = []
    n_list  = []
    minDocFreq_list  = []
    smoothing_list = []

    metrics = model_obj.avgMetrics
    assert type(metrics) is list
    assert len(metrics) > 0

    for x in range(len(model_obj._paramMap.keys())):
        if list(model_obj._paramMap.keys())[x].name=='estimatorParamMaps':
            param_map_key = list(model_obj._paramMap.keys())[x]

    params = model_obj._paramMap[param_map_key]

    for i in range(len(params)):
        for k in params[i].keys():
            if k.name =='n':
                n_list.append(params[i][k])
            if k.name =='smoothing':
                smoothing_list.append(params[i][k])
            if k.name =='minDocFreq':
                minDocFreq_list.append(params[i][k])

    results_df =  pd.DataFrame({
             'model':'naive_bayes',
             'minDocFreq': minDocFreq_list, 
             'n': n_list,
             'smoothing': smoothing_list,
             'metrics':metrics
              })

    spark_version = [int(x) for x in sc.version.split('.')]

    if spark_version[0] <= 2:
        if spark_version[1] < 1:
            results_df.metrics = 1.0*results_df['metrics'] / cv_fold

    return results_df

In [0]:
table2 = hyperparameter_getter(model_nb)

In [0]:
tk = Tokenizer(inputCol= "Comment", outputCol = "tokens")
gram = NGram(inputCol="tokens", outputCol="gram")
tf     = HashingTF(inputCol="gram", outputCol="rawFeatures", numFeatures=2e5)
idf     = IDF(inputCol="rawFeatures", outputCol="features")
rf      = RandomForestClassifier()

random_forest_pipeline  = Pipeline(stages= [tk, gram, tf, idf, rf])
paramGrid_rf = ParamGridBuilder() \
                        .addGrid(gram.n, [1,2,3]) \
                        .addGrid(idf.minDocFreq, [1, 2, 5]) \
                        .addGrid(rf.maxDepth, [3,6,12]) \
                        .addGrid(rf.numTrees, [10,50,100]) \
                        .build()
crossval = CrossValidator(estimator          = random_forest_pipeline,
                          estimatorParamMaps = paramGrid_rf,
                          evaluator          = MulticlassClassificationEvaluator(),
                          numFolds           = 2, collectSubModels=True)
model_rf    = crossval.fit(train) 

In [0]:
def hyperparameter_getter(model_obj,cv_fold = 2.0):

    n_list  = []
    minDocFreq_list  = []
    maxDepth_list =[]
    numTrees_list = []

    metrics = model_obj.avgMetrics
    assert type(metrics) is list
    assert len(metrics) > 0

    for x in range(len(model_obj._paramMap.keys())):
        if list(model_obj._paramMap.keys())[x].name=='estimatorParamMaps':
            param_map_key = list(model_obj._paramMap.keys())[x]

    params = model_obj._paramMap[param_map_key]

    for i in range(len(params)):
        for k in params[i].keys():
            if k.name =='n':
                n_list.append(params[i][k])
            if k.name =='numTrees':
                numTrees_list.append(params[i][k])
            if k.name =='minDocFreq':
                minDocFreq_list.append(params[i][k])
            if k.name =='maxDepth':
                maxDepth_list.append(params[i][k])

    results_df =  pd.DataFrame({
        'model':'random_forest',
        'minDocFreq': minDocFreq_list,
        'n': n_list,
        'numTrees': numTrees_list,
        'maxDepth':maxDepth_list,
        'metrics':metrics
        })

    spark_version = [int(x) for x in sc.version.split('.')]

    if spark_version[0] <= 2:
        if spark_version[1] < 1:
            results_df.metrics = 1.0*results_df['metrics'] / cv_fold

    return results_df

In [0]:
table3 = hyperparameter_getter(model_rf)

In [0]:
trainingResultsRDD=spark.createDataFrame(trainingResults)
table2RDD=spark.createDataFrame(table2)
table3RDD=spark.createDataFrame(table3)
allTrainingResults = trainingResultsRDD.unionByName(table2RDD, allowMissingColumns=True)
allTrainingResults = allTrainingResults.unionByName(table3RDD, allowMissingColumns=True)
sqlContext.registerDataFrameAsTable(allTrainingResults, "allTrainingResults")
top5Models = spark.sql("SELECT * FROM allTrainingResults a ORDER BY a.metrics DESC LIMIT 5");
display(top5Models)
bestByModel = spark.sql("SELECT DISTINCT model, max(metrics) FROM allTrainingResults a GROUP BY a.model");
display(bestByModel)


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1545275712153853> in <module>
      1 trainingResultsRDD=spark.createDataFrame(trainingResults)
----> 2 table2RDD=spark.createDataFrame(table2)
      3 table3RDD=spark.createDataFrame(table3)
      4 allTrainingResults = trainingResultsRDD.unionByName(table2RDD, allowMissingColumns=True)
      5 allTrainingResults = allTrainingResults.unionByName(table3RDD, allowMissingColumns=True)

NameError: name 'table2' is not defined

In [0]:
sentimentModel = model.bestModel

In [0]:
prediction = sentimentModel.transform(test)

In [0]:
display(prediction)

Comment,label,tokens,gram,rawFeatures,features,rawPrediction,probability,prediction
,1.0,List(),List(),"Map(vectorType -> sparse, length -> 200000, indices -> List(193372), values -> List(1.0))","Map(vectorType -> sparse, length -> 200000, indices -> List(193372), values -> List(0.3544856704998812))","Map(vectorType -> dense, length -> 3, values -> List(0.4041678343303417, 0.7904273611832181, -1.1945951955135599))","Map(vectorType -> dense, length -> 3, values -> List(0.3740257211356539, 0.5503663211592151, 0.07560795770513092))",1.0
,1.0,List(),List(),"Map(vectorType -> sparse, length -> 200000, indices -> List(193372), values -> List(1.0))","Map(vectorType -> sparse, length -> 200000, indices -> List(193372), values -> List(0.3544856704998812))","Map(vectorType -> dense, length -> 3, values -> List(0.4041678343303417, 0.7904273611832181, -1.1945951955135599))","Map(vectorType -> dense, length -> 3, values -> List(0.3740257211356539, 0.5503663211592151, 0.07560795770513092))",1.0
,1.0,List(),List(),"Map(vectorType -> sparse, length -> 200000, indices -> List(193372), values -> List(1.0))","Map(vectorType -> sparse, length -> 200000, indices -> List(193372), values -> List(0.3544856704998812))","Map(vectorType -> dense, length -> 3, values -> List(0.4041678343303417, 0.7904273611832181, -1.1945951955135599))","Map(vectorType -> dense, length -> 3, values -> List(0.3740257211356539, 0.5503663211592151, 0.07560795770513092))",1.0
,2.0,List(),List(),"Map(vectorType -> sparse, length -> 200000, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 200000, indices -> List(), values -> List())","Map(vectorType -> dense, length -> 3, values -> List(0.4097945778632302, 0.7805116922440584, -1.1903062701072886))","Map(vectorType -> dense, length -> 3, values -> List(0.37726604807224917, 0.5465729315360023, 0.07616102039174852))",1.0
christian movie but that one favorite christian song ever,1.0,"List(, , , , , christian, movie, but, that, , one, , , favorite, christian, song, ever)","List(, , , , , christian, movie, but, that, , one, , , favorite, christian, song, ever)","Map(vectorType -> sparse, length -> 200000, indices -> List(29760, 57350, 71086, 85594, 131439, 133885, 170815, 172605, 193372), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 8.0))","Map(vectorType -> sparse, length -> 200000, indices -> List(29760, 57350, 71086, 85594, 131439, 133885, 170815, 172605, 193372), values -> List(2.790886206681359, 4.495634298919785, 0.0, 4.629165691544307, 1.838877392205125, 10.377562958959459, 3.2792389745952915, 3.2077800106131464, 2.83588536399905))","Map(vectorType -> dense, length -> 3, values -> List(2.439390948050258, -1.0604033790677447, -1.3789875689825104))","Map(vectorType -> dense, length -> 3, values -> List(0.9504194855753102, 0.028706085074097172, 0.020874429350592617))",0.0
body tell rambo nothing,0.0,"List(, , , , body, tell, rambo, nothing)","List(, , , , body, tell, rambo, nothing)","Map(vectorType -> sparse, length -> 200000, indices -> List(44292, 96950, 160841, 181850, 193372), values -> List(1.0, 1.0, 1.0, 1.0, 4.0))","Map(vectorType -> sparse, length -> 200000, indices -> List(44292, 96950, 160841, 181850, 193372), values -> List(0.0, 5.476463551931511, 5.476463551931511, 4.783316371371566, 1.417942681999525))","Map(vectorType -> dense, length -> 3, values -> List(-1.0645275472268425, 2.0441374971696584, -0.9796099499428155))","Map(vectorType -> dense, length -> 3, values -> List(0.0408500726447444, 0.9146794207325654, 0.04447050662269009))",1.0
way,2.0,"List(, , , , way)","List(, , , , way)","Map(vectorType -> sparse, length -> 200000, indices -> List(189391, 193372), values -> List(1.0, 4.0))","Map(vectorType -> sparse, length -> 200000, indices -> List(189391, 193372), values -> List(4.629165691544307, 1.417942681999525))","Map(vectorType -> dense, length -> 3, values -> List(-0.0072654170987404565, 0.14588565655947405, -0.1386202394607332))","Map(vectorType -> dense, length -> 3, val

In [0]:
tfidf_results = {}
predictedAndLabels = prediction.select(["prediction","label"]).rdd.map(lambda r: (float(r[0]), float(r[1])))
tfidfmetrics = MulticlassMetrics(predictedAndLabels)
tfidf_metrics = []
tfidf_metrics.append(tfidfmetrics.weightedFMeasure())
tfidf_metrics.append(tfidfmetrics.weightedPrecision)
tfidf_metrics.append(tfidfmetrics.weightedRecall)
tfidf_results.update({'LogisticRegression' : tfidf_metrics})
index = ['f1_measure', 'precision', 'recall' ]
n2_df = pd.DataFrame(tfidf_results, index=index)
print("Model results with no hyperparameter tuning using bigrams as features")
n2_df.head()


/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
Model results with no hyperparameter tuning using bigrams as features


,LogisticRegression
f1_measure,0.634082
precision,0.639193
recall,0.648649


In [0]:
allMovieFile = cs.createAllMovieDf('/FileStore/shared_uploads/isaiah.williams@ucalgary.ca/')
allMovieFile = cs.preProcess(allMovieFile)

In [0]:
allMovieFile = allMovieFile.withColumn("date",to_date(split(col("Time"),"T")[0],"yyyy-MM-dd"))

In [0]:
movieMetaData = spark.read.option("header", True).option("wholeFile", True).option("escape", "\"").option("multiLine", True).csv('/FileStore/xlsxYoutubeData_and_MovieMetaData__1_-1.csv')
movieMetaData = movieMetaData.select('*').withColumnRenamed("Movie Title", "Movie_Title")
movieMetaData = movieMetaData.withColumn("date", expr("make_date(ReleaseYear, ReleaseMonth, ReleaseDay)"))

In [0]:
sqlContext.registerDataFrameAsTable(allMovieFile, "allMovieFile")
sqlContext.registerDataFrameAsTable(movieMetaData, "movieMetaData")
allMovieFilteredComments = spark.sql("SELECT a.*, m.Movie_Title AS metaDataMovie, m.date AS metaDataDate FROM allMovieFile a, movieMetaData m WHERE a.MovieName = m.Movie_Title AND a.date<m.date")

In [0]:
commentSentiment = sentimentModel.transform(allMovieFilteredComments)

In [0]:
finalCommentSentiment = commentSentiment.select("MovieName", "prediction")

In [0]:
finalCommentSentimentGrouped = finalCommentSentiment.groupBy("MovieName").avg( "prediction")
display(finalCommentSentimentGrouped)